In [1]:
import torch

Load SD2.1 as PyTorch state dict (SDXL is only available as safetensors, not pytorch statedict)

In [10]:
sdict = torch.load('../../../../.hf-cache/CVL-Heidelberg/sd21_encD_canny_14m.ckpt', map_location=torch.device('cpu'))['state_dict']

In [2]:
from collections import defaultdict




nested_dict = to_nested_dict(lst)

import json
print(json.dumps(nested_dict, indent=4))

{
    "scale_list": {},
    "control_model": {
        "time_embed": {},
        "input_blocks": {
            "level1": {
                "level2": {}
            }
        },
        "middle_block": {
            "level1": {
                "level2": {
                    "level3": {}
                }
            }
        }
    }
}


In [118]:
print_state_dict(contains='time')

control_model.time_embed.0.weight
control_model.time_embed.0.bias
control_model.time_embed.2.weight
control_model.time_embed.2.bias


In [119]:
sdict['control_model.time_embed.0.weight'].shape, sdict['control_model.time_embed.0.bias'].shape

(torch.Size([1280, 320]), torch.Size([1280]))

In [120]:
sdict['control_model.time_embed.2.weight'].shape, sdict['control_model.time_embed.2.bias'].shape

(torch.Size([1280, 1280]), torch.Size([1280]))

In [121]:
print_state_dict(lv=0)

scale_list
control_model
enc_zero_convs_out
enc_zero_convs_in
dec_zero_convs_out
middle_block_out
input_hint_block


In [122]:
print_state_dict(contains='control_model', lv=1)

control_model.time_embed
control_model.input_blocks
control_model.middle_block


In [123]:
print_state_dict(contains='control_model.input_blocks.1.*')

control_model.input_blocks.1.0.in_layers.0.weight
control_model.input_blocks.1.0.in_layers.0.bias
control_model.input_blocks.1.0.in_layers.2.weight
control_model.input_blocks.1.0.in_layers.2.bias
control_model.input_blocks.1.0.emb_layers.1.weight
control_model.input_blocks.1.0.emb_layers.1.bias
control_model.input_blocks.1.0.out_layers.0.weight
control_model.input_blocks.1.0.out_layers.0.bias
control_model.input_blocks.1.0.out_layers.3.weight
control_model.input_blocks.1.0.out_layers.3.bias
control_model.input_blocks.1.0.skip_connection.weight
control_model.input_blocks.1.0.skip_connection.bias
control_model.input_blocks.1.1.norm.weight
control_model.input_blocks.1.1.norm.bias
control_model.input_blocks.1.1.proj_in.weight
control_model.input_blocks.1.1.proj_in.bias
control_model.input_blocks.1.1.transformer_blocks.0.attn1.to_q.weight
control_model.input_blocks.1.1.transformer_blocks.0.attn1.to_k.weight
control_model.input_blocks.1.1.transformer_blocks.0.attn1.to_v.weight
control_model.

In [124]:
resblock = get_state_dict(contains='control_model.input_blocks.1.0.*')

In [150]:
def print_shapes(contains, full=False):
    objs = get_state_dict(contains)
    for m in objs:
        if full: print(f"{m:<60} {str(sdict[m].shape)}")
        else: print(f"{m.replace(contains,''):<25} {str(sdict[m].shape)}")

In [160]:
print_shapes('control_model.input_blocks.0', full=True)

control_model.input_blocks.0.0.weight                        torch.Size([4, 4, 3, 3])
control_model.input_blocks.0.0.bias                          torch.Size([4])


In [156]:
print_shapes('control_model.input_blocks.1.0.')

in_layers.0.weight        torch.Size([324])
in_layers.0.bias          torch.Size([324])
in_layers.2.weight        torch.Size([4, 324, 3, 3])
in_layers.2.bias          torch.Size([4])
emb_layers.1.weight       torch.Size([4, 1280])
emb_layers.1.bias         torch.Size([4])
out_layers.0.weight       torch.Size([4])
out_layers.0.bias         torch.Size([4])
out_layers.3.weight       torch.Size([4, 4, 3, 3])
out_layers.3.bias         torch.Size([4])
skip_connection.weight    torch.Size([4, 324, 1, 1])
skip_connection.bias      torch.Size([4])


In [159]:
print_shapes('control_model.input_blocks.2.0.')

in_layers.0.weight        torch.Size([324])
in_layers.0.bias          torch.Size([324])
in_layers.2.weight        torch.Size([4, 324, 3, 3])
in_layers.2.bias          torch.Size([4])
emb_layers.1.weight       torch.Size([4, 1280])
emb_layers.1.bias         torch.Size([4])
out_layers.0.weight       torch.Size([4])
out_layers.0.bias         torch.Size([4])
out_layers.3.weight       torch.Size([4, 4, 3, 3])
out_layers.3.bias         torch.Size([4])
skip_connection.weight    torch.Size([4, 324, 1, 1])
skip_connection.bias      torch.Size([4])


In [164]:
print_shapes('control_model.input_blocks.3.')

0.op.weight               torch.Size([4, 324, 3, 3])
0.op.bias                 torch.Size([4])


In [167]:
print_state_dict('control_model.input_blocks',lv=2)

control_model.input_blocks.0
control_model.input_blocks.1
control_model.input_blocks.2
control_model.input_blocks.3
control_model.input_blocks.4
control_model.input_blocks.5
control_model.input_blocks.6
control_model.input_blocks.7
control_model.input_blocks.8
control_model.input_blocks.9
control_model.input_blocks.10
control_model.input_blocks.11


In [169]:
print_shapes('control_model.input_blocks.4.0.')

in_layers.0.weight        torch.Size([324])
in_layers.0.bias          torch.Size([324])
in_layers.2.weight        torch.Size([8, 324, 3, 3])
in_layers.2.bias          torch.Size([8])
emb_layers.1.weight       torch.Size([8, 1280])
emb_layers.1.bias         torch.Size([8])
out_layers.0.weight       torch.Size([8])
out_layers.0.bias         torch.Size([8])
out_layers.3.weight       torch.Size([8, 8, 3, 3])
out_layers.3.bias         torch.Size([8])
skip_connection.weight    torch.Size([8, 324, 1, 1])
skip_connection.bias      torch.Size([8])


In [170]:
print_shapes('control_model.input_blocks.5.0.')

in_layers.0.weight        torch.Size([648])
in_layers.0.bias          torch.Size([648])
in_layers.2.weight        torch.Size([8, 648, 3, 3])
in_layers.2.bias          torch.Size([8])
emb_layers.1.weight       torch.Size([8, 1280])
emb_layers.1.bias         torch.Size([8])
out_layers.0.weight       torch.Size([8])
out_layers.0.bias         torch.Size([8])
out_layers.3.weight       torch.Size([8, 8, 3, 3])
out_layers.3.bias         torch.Size([8])
skip_connection.weight    torch.Size([8, 648, 1, 1])
skip_connection.bias      torch.Size([8])
